# Tools

#### A tool is just a python function (or API) that is packaged in a way the LLM can understand and call when needed.

- Built-in Tools
- Custom Tools

How tools fits into the Agents ecosystem?  
- An Ai Agent is an LLM powered-system that can autonomously think, decide, and take action using external tools and APIs to achieve goal.

## Built-in Tool: DuckDuckGo Search

In [18]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("Who won the FIFA World Cup 2022?")

"The 2022 FIFA World Cup was the 22nd FIFA World Cup , the quadrennial world championship for national football teams organised by FIFA . It took place in Qatar from 20 November to 18 December 2022 , after the country was awarded the hosting rights in 2010. The FIFA World Cup 2022 Qatar will see 32 teams compete to win . Get World Cup results, football scores, group standings, points tables and more. Watch the highlights from the match between Argentina and France played at Lusail Stadium, Lusail on Sunday, 18 December 2022 . FIFA World Cup .Messi and Mbappe duked it out, but ultimately it was the former and Argentina who came out on top. Thank you for joining us today and for the last four weeks of the 2022 World Cup . I'll leave you with the best bits from a stunning final for the ages... Jairzinho is the only player in FIFA World Cup history to score in every single game of the tournament. He did so while guiding the greatest football team of all time to a FIFA World Cup triumph.Mes

In [19]:
print(search.name)
print(search.description)
print(search.args)

duckduckgo_search
A wrapper around DuckDuckGo Search. Useful for when you need to answer questions about current events. Input should be a search query.
{'query': {'description': 'search query to look up', 'title': 'Query', 'type': 'string'}}


## Built-in Tool: Shell Tool

In [21]:
from langchain_community.tools import ShellTool

shellTool = ShellTool()

print(shellTool.invoke('ls -l'))

Executing command:
 ls -l
total 1016
-rw-r--r--@ 1 bytescode  staff   28328 Jan  6 17:27 Chains.ipynb
-rw-r--r--@ 1 bytescode  staff     556 Jan  5 12:31 ChatBot.py
-rw-r--r--@ 1 bytescode  staff   10437 Jan  5 13:06 ChatModel_Gemini.ipynb
-rw-r--r--@ 1 bytescode  staff   22939 Jan  9 00:21 ChatModel_HuggingFace.ipynb
-rw-r--r--@ 1 bytescode  staff  294510 Jan  8 15:18 Document_Loaders.ipynb
-rw-r--r--@ 1 bytescode  staff    6072 Jan  4 15:29 Embedding_Gemini.ipynb
-rw-r--r--@ 1 bytescode  staff   13861 Jan  6 15:02 Output_Parsers.ipynb
-rw-r--r--@ 1 bytescode  staff    1350 Jan  5 11:32 Prompt.py
drwxr-xr-x@ 7 bytescode  staff     224 Jan  8 13:26 Story_Book
-rw-r--r--@ 1 bytescode  staff   18074 Jan  5 17:52 Structured_Output.ipynb
-rw-r--r--@ 1 bytescode  staff    8593 Jan  8 17:54 Text_Splitters.ipynb
-rw-r--r--@ 1 bytescode  staff    7318 Jan 10 12:27 Tools_ToolCalling.ipynb
-rw-r--r--@ 1 bytescode  staff    8745 Jan  8 18:51 Vector_store.ipynb
-rw-r--r--@ 1 bytescode  staff   609

In [22]:
print(shellTool.name)
print(shellTool.description)
print(shellTool.args)

terminal
Run shell commands on this MacOS machine.
{'commands': {'anyOf': [{'type': 'string'}, {'items': {'type': 'string'}, 'type': 'array'}], 'description': 'List of shell commands to run. Deserialized using json.loads', 'title': 'Commands'}}


## Making Custom Tool

- **Step 1.** Create a function
- **Step 2.** Add type hint
- **Step 3.** Add tool decorator

In [12]:
from langchain_community.tools import tool

@tool
def multiply(a:int, b:int) -> int:
    """Multiply two numbers."""
    return a*b

multiply.invoke({'a':4, 'b':7})

28

In [13]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [24]:
multiply.args_schema.model_json_schema()

{'description': 'Multiply two numbers.',
 'properties': {'a': {'title': 'A', 'type': 'integer'},
  'b': {'title': 'B', 'type': 'integer'}},
 'required': ['a', 'b'],
 'title': 'multiply',
 'type': 'object'}

In [1]:
# Using pydantic & StructuredTool

from langchain_community.tools import StructuredTool
from pydantic import BaseModel, Field

class MulInput(BaseModel):
    a: int = Field(description='First Number')
    b: int = Field(description='Second Number')

def mul_func(a:int, b:int) -> int:
    return a*b

mulTool = StructuredTool.from_function(
    func=mul_func,
    name='Multiply',
    description='Multiply two given number',
    args_schema=MulInput,
)

mulTool.invoke({'a':4, 'b':7})

28

In [33]:
print(mulTool.name)
print(mulTool.description)
print(mulTool.args)

Multiply
Multiply two given number
{'a': {'description': 'First Number', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'Second Number', 'title': 'B', 'type': 'integer'}}


# Tool-Calling
- Tool Binding
- Tool Calling
- Tool Execution

**Tool Calling** is the process where LLM decides, during a conversation or task, that is need to use  specific tool and generates a structured output with:
- the name of the tool
- and the arguments to call it with

***Note:*** The **LLM doest not actually run the tool** -it just suggest the tool and arguments.  
The **actual execution is handled by LangChain or You.**

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

Gemini = ChatGoogleGenerativeAI(model='gemini-2.5-flash')

# Tool Binding
GeminiTools = Gemini.bind_tools([mulTool])

/Users/bytescode/My_Skill/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
result = GeminiTools.invoke('Hey, can you do multiplication of 7 and 9 for me?')
result.tool_calls

[{'name': 'Multiply',
  'args': {'a': 7, 'b': 9},
  'id': 'af4a6121-d620-4b72-8a52-8c23717d55a1',
  'type': 'tool_call'}]

**Tool Execution :** Is the process of calling a tool and executing it within the context of a larger workflow or application.


In [45]:
# Tool Execution
from langchain_core.messages import HumanMessage

question = HumanMessage('Hey, can you do multiplication of 7 and 9 for me?')

messages = [question]

In [46]:
result = GeminiTools.invoke(messages)
messages.append(result)
messages

[HumanMessage(content='Hey, can you do multiplication of 7 and 9 for me?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'Multiply', 'arguments': '{"b": 9, "a": 7}'}, '__gemini_function_call_thought_signatures__': {'44963c17-a1ec-4c57-8f21-bae6ef8eb6f5': 'CsoBAXLI2ny7vt8QwY82Zpry4t3vkxNeSNcEYz/rDdDGQHWXx8vWFhqfgXD61fnXc8UFsSvSRzXeIzghhwUILKSJKqK+ln1x6jHwoJJK/9R2/K2OgU/KnQak3N01TSq8I+hplXE7jLidupwfDbsw1E2BhJqestZ2DAQLMfXPut+hfTQxcx81eVFWHwv6BBXSSJkPsmsZA9KJdqFxpjPTgk6fgKdZRUTw+SvvTOQxqJPbmoGVeuApaWbdc6qRCQTGHoBKcQywKcIDJ+WjOg=='}}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019ba724-86a7-7013-804d-4777eb962c6d-0', tool_calls=[{'name': 'Multiply', 'args': {'b': 9, 'a': 7}, 'id': '44963c17-a1ec-4c57-8f21-bae6ef8eb6f5', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 63, 'output_tokens': 65, 

In [48]:
ans = mulTool.invoke(result.tool_calls[0])
messages.append(ans)

In [49]:
res = GeminiTools.invoke(messages)

In [51]:
res.content

'The multiplication of 7 and 9 is 63.'

In [52]:
# Code in one cell

question = HumanMessage('Hey, can you do multiplication of 21 and 9 for me?')
messages = [question]

result = GeminiTools.invoke(messages)
messages.append(result)

ans = mulTool.invoke(result.tool_calls[0])
messages.append(ans)

res = GeminiTools.invoke(messages)
res.content

'The multiplication of 21 and 9 is 189.'